# Ridge Regression Alpha

In [1]:
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import statsmodels.api as sm
import numpy as np

In [2]:
# Import Dataframe of Factors
df_Macro_Factors = pd.read_excel(r'./macro_df.xlsx',index_col='DATE')
df_Macro_Factors

,Mkt-RF,SMB,HML,RF,GS2,GS10,CPIAUCSL,PPIACO,CPILFESL,EXUSEU
DATE,,,,,,,,,,
1999-01-01,3.50,0.75,-4.60,0.35,4.62,4.72,164.700,122.900,175.600,1.1591
1999-02-01,-4.08,-6.08,1.92,0.35,4.88,5.00,164.700,122.300,175.600,1.1203
1999-03-01,3.45,-3.80,-2.74,0.43,5.05,5.23,164.800,122.600,175.700,1.0886
1999-04-01,4.33,3.91,2.46,0.37,4.98,5.18,165.900,123.600,176.300,1.0701
1999-05-01,-2.46,3.34,2.35,0.34,5.25,5.54,166.000,124.700,176.500,1.0630
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,-2.39,-3.16,-1.06,0.45,4.90,4.17,306.269,257.680,309.661,1.0910
2023-09-01,-5.24,-2.51,1.52,0.43,5.02,4.38,307.481,258.859,310.661,1.0672
2023-10-01,-3.19,-3.87,0.19,0.47,5.07,4.80,307.619,255.042,311.365,1.0565


In [3]:
# Import Dataframe of technical data
df_Technical_Factors = pd.read_excel(r'./Technical_data_download.xlsx',index_col='DATE')
df_Technical_Factors

,^VIX,Volume,SMA_50,SMA_200,RSI
DATE,,,,,
1998-10-01,36.608182,8.182568e+08,1037.751182,1071.411241,52.240208
1998-11-01,26.223000,6.725640e+08,1051.228810,1082.556657,70.783049
1998-12-01,25.479091,6.900659e+08,1114.600713,1097.654393,61.586491
1999-01-01,28.035263,8.533421e+08,1184.945938,1112.406447,60.814687
1999-02-01,28.822632,7.660979e+08,1224.484461,1125.385239,51.777563
...,...,...,...,...,...
2023-08-01,15.853913,3.775688e+09,4439.430929,4123.123807,47.675647
2023-09-01,15.241000,3.674149e+09,4475.096174,4181.626158,44.397154
2023-10-01,18.887727,3.796339e+09,4399.047676,4223.552843,40.547568


In [4]:
# Merging macro Dataframe and Technical Dataframe
Total_Factors = pd.merge(df_Macro_Factors,df_Technical_Factors,left_index=True, right_index=True).dropna()
Total_Factors

,Mkt-RF,SMB,HML,RF,GS2,GS10,CPIAUCSL,PPIACO,CPILFESL,EXUSEU,^VIX,Volume,SMA_50,SMA_200,RSI
DATE,,,,,,,,,,,,,,,
1999-01-01,3.50,0.75,-4.60,0.35,4.62,4.72,164.700,122.900,175.600,1.1591,28.035263,8.533421e+08,1184.945938,1112.406447,60.814687
1999-02-01,-4.08,-6.08,1.92,0.35,4.88,5.00,164.700,122.300,175.600,1.1203,28.822632,7.660979e+08,1224.484461,1125.385239,51.777563
1999-03-01,3.45,-3.80,-2.74,0.43,5.05,5.23,164.800,122.600,175.700,1.0886,25.309565,7.827174e+08,1252.149560,1141.556845,56.689673
1999-04-01,4.33,3.91,2.46,0.37,4.98,5.18,165.900,123.600,176.300,1.0701,23.478571,8.820571e+08,1280.993906,1163.346467,58.550735
1999-05-01,-2.46,3.34,2.35,0.34,5.25,5.54,166.000,124.700,176.500,1.0630,26.204500,7.940130e+08,1318.071762,1182.959364,49.815232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,-2.39,-3.16,-1.06,0.45,4.90,4.17,306.269,257.680,309.661,1.0910,15.853913,3.775688e+09,4439.430929,4123.123807,47.675647
2023-09-01,-5.24,-2.51,1.52,0.43,5.02,4.38,307.481,258.859,310.661,1.0672,15.241000,3.674149e+09,4475.096174,4181.626158,44.397154
2023-10-01,-3.19,-3.87,0.19,0.47,5.07,4.80,307.619,255.042,311.365,1.0565,18.887727,3.796339e+09,4399.047676,4223.552843,40.547568


In [5]:
# Import Etf prices Dataframe
ETF_data = pd.read_excel(r'./ETF_Prices.xlsx',index_col='Date')
ETF_data.index.name = 'DATE'
ETF_data.index = pd.to_datetime(ETF_data.index)

# Converting Etf prices Dataframe to monthly data period
ETF_data = ETF_data.groupby(ETF_data.index.to_period('M')).first()
ETF_data

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
DATE,,,,,,,,,
2000-01,25.968750,26.562500,18.582453,28.843750,55.437500,22.750000,27.453125,30.296875,30.125000
2000-02,23.750000,27.250000,18.836311,27.000000,51.625000,23.000000,27.437500,30.500000,28.187500
2000-03,20.765625,26.812500,16.957758,25.546875,56.437500,20.328125,25.343750,28.375000,25.390625
2000-04,23.796875,29.812500,20.664095,29.328125,58.000000,22.187500,26.906250,30.656250,30.140625
2000-05,22.250000,29.343750,19.458265,30.312500,55.781250,22.625000,28.484375,30.406250,29.531250
...,...,...,...,...,...,...,...,...,...
2023-08,85.339996,87.070000,35.330002,110.750000,178.649994,75.339996,66.250000,133.449997,171.770004
2023-09,83.820000,90.739998,34.689999,108.800003,176.199997,72.139999,62.639999,133.570007,169.669998
2023-10,77.510002,88.589996,32.889999,100.389999,165.690002,68.339996,56.189999,128.559998,161.110001


In [6]:
# Converting Etf prices to Etf percentage change
ETF_data_returns = ETF_data.pct_change()[1:]
ETF_data_returns.index = ETF_data_returns.index.to_timestamp()
ETF_data_returns

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
DATE,,,,,,,,,
2000-02-01,-0.085439,0.025882,0.013661,-0.063922,-0.068771,0.010989,-0.000569,0.006704,-0.064315
2000-03-01,-0.125658,-0.016055,-0.099730,-0.053819,0.093220,-0.116168,-0.076310,-0.069672,-0.099224
2000-04-01,0.145974,0.111888,0.218563,0.148012,0.027685,0.091468,0.061652,0.080396,0.187077
2000-05-01,-0.065003,-0.015723,-0.058354,0.033564,-0.038254,0.019718,0.058653,-0.008155,-0.020218
2000-06-01,-0.027388,0.089457,0.080235,-0.019072,-0.076751,0.060773,0.004937,-0.035971,-0.075132
...,...,...,...,...,...,...,...,...,...
2023-08-01,0.025968,0.070182,0.042183,0.031768,0.029742,0.007893,0.004854,0.013750,-0.000524
2023-09-01,-0.017811,0.042150,-0.018115,-0.017607,-0.013714,-0.042474,-0.054491,0.000899,-0.012226
2023-10-01,-0.075280,-0.023694,-0.051888,-0.077298,-0.059648,-0.052675,-0.102969,-0.037508,-0.050451


In [7]:
# Merging all previous Dataframes
Regression_df = pd.merge(ETF_data_returns,Total_Factors,left_index=True, right_index=True).dropna()
Regression_df

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,Mkt-RF,...,GS10,CPIAUCSL,PPIACO,CPILFESL,EXUSEU,^VIX,Volume,SMA_50,SMA_200,RSI
DATE,,,,,,,,,,,,,,,,,,,,,
2000-02-01,-0.085439,0.025882,0.013661,-0.063922,-0.068771,0.010989,-0.000569,0.006704,-0.064315,2.45,...,6.52,170.000,129.800,179.400,0.9834,23.595500,1.045600e+09,1422.873320,1360.760034,43.998678
2000-03-01,-0.125658,-0.016055,-0.099730,-0.053819,0.093220,-0.116168,-0.076310,-0.069672,-0.099224,5.20,...,6.26,171.000,130.800,180.000,0.9643,22.718261,1.137226e+09,1408.742886,1367.743448,57.480524
2000-04-01,0.145974,0.111888,0.218563,0.148012,0.027685,0.091468,0.061652,0.080396,0.187077,-6.40,...,5.99,170.900,130.700,180.300,0.9449,27.164211,1.058235e+09,1427.906636,1382.617432,51.252927
2000-05-01,-0.065003,-0.015723,-0.058354,0.033564,-0.038254,0.019718,0.058653,-0.008155,-0.020218,-4.42,...,6.44,171.200,131.600,180.700,0.9059,26.373182,9.044682e+08,1448.358141,1389.461669,46.151833
2000-06-01,-0.027388,0.089457,0.080235,-0.019072,-0.076751,0.060773,0.004937,-0.035971,-0.075132,4.64,...,6.10,172.200,133.800,181.100,0.9505,21.540000,9.881045e+08,1442.721109,1401.175454,54.047348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.025968,0.070182,0.042183,0.031768,0.029742,0.007893,0.004854,0.013750,-0.000524,-2.39,...,4.17,306.269,257.680,309.661,1.0910,15.853913,3.775688e+09,4439.430929,4123.123807,47.675647
2023-09-01,-0.017811,0.042150,-0.018115,-0.017607,-0.013714,-0.042474,-0.054491,0.000899,-0.012226,-5.24,...,4.38,307.481,258.859,310.661,1.0672,15.241000,3.674149e+09,4475.096174,4181.626158,44.397154
2023-10-01,-0.075280,-0.023694,-0.051888,-0.077298,-0.059648,-0.052675,-0.102969,-0.037508,-0.050451,-3.19,...,4.80,307.619,255.042,311.365,1.0565,18.887727,3.796339e+09,4399.047676,4223.552843,40.547568


In [8]:
# Defining Ridge Model
def Compute_Alpha(y, x, theta_head):
    theta_head = theta_head[1:]
    x = x[:, 1:]

    mean_y = np.mean(y)
    mean_y_pred = np.mean(x, axis=0) @ theta_head
    alpha = mean_y - mean_y_pred

    return alpha

def Alpha_Calculation(Regression_df, Target_Y, Factor):
    y = Regression_df[Target_Y].values
    x = Regression_df[Factor].values
    x = sm.add_constant(x)

    # Scale
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

    # Parameters of Ridge Regression
    alpha_value = 1.0
    ridge_model = Ridge(alpha=1.0)

    # Model scaled
    ridge_model.fit(x_scaled, y)

    theta_head = ridge_model.coef_
    return Compute_Alpha(y, x_scaled, theta_head)

In [9]:
# Ridge model example
Target_Y = 'XLB'
Factor = Total_Factors.columns

Alpha_Calculation(Regression_df, Target_Y, Factor)

0.006065098851786403

In [10]:
# Computing Dataframe with Ridge Model
df_all_alpha = pd.DataFrame()

for regression_df in Regression_df.rolling(window=60):

    if regression_df.shape[0] >= 60:
        alpha_list = []
        index_start = regression_df.index[0]
        index_end = regression_df.index[-1]
        
        for target_y in ETF_data.columns.to_list():
            alpha = Alpha_Calculation(Regression_df = regression_df, Target_Y = target_y, Factor = Total_Factors.columns)
            alpha_list.append(alpha)

        df_alpha = pd.DataFrame(alpha_list,index=ETF_data.columns.to_list(),columns=[index_end]).T
        

        df_all_alpha = pd.concat([df_all_alpha,df_alpha],axis=0)

df_all_alpha

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
2005-01-01,0.003852,0.006318,0.006061,0.002557,-0.011051,0.001011,0.001155,0.000940,0.004219
2005-02-01,0.005066,0.007416,0.005707,0.003347,-0.010614,0.001030,0.001831,0.000655,0.004697
2005-03-01,0.008348,0.009783,0.007242,0.004442,-0.012043,0.003073,0.003394,0.002291,0.006434
2005-04-01,0.005348,0.008279,0.002509,0.001698,-0.013165,0.001045,0.002435,0.000528,0.002787
2005-05-01,0.005417,0.007535,0.003785,0.000773,-0.012991,0.000826,0.001959,0.001469,0.002267
...,...,...,...,...,...,...,...,...,...
2023-08-01,0.008308,0.008803,0.006565,0.008531,0.017963,0.006723,0.005438,0.007914,0.010261
2023-09-01,0.008085,0.009893,0.005953,0.007983,0.016861,0.005830,0.004129,0.007350,0.009086
2023-10-01,0.006929,0.008935,0.005545,0.006251,0.015782,0.004896,0.002798,0.006077,0.008289
2023-11-01,0.007885,0.010224,0.006159,0.007649,0.017161,0.004295,0.003727,0.006601,0.008882


In [11]:
# Manipulating index and renaming Dataframe in order to save it as xlsx
df_ridge_alpha = df_all_alpha.rename_axis('DATE')
df_ridge_alpha

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
DATE,,,,,,,,,
2005-01-01,0.003852,0.006318,0.006061,0.002557,-0.011051,0.001011,0.001155,0.000940,0.004219
2005-02-01,0.005066,0.007416,0.005707,0.003347,-0.010614,0.001030,0.001831,0.000655,0.004697
2005-03-01,0.008348,0.009783,0.007242,0.004442,-0.012043,0.003073,0.003394,0.002291,0.006434
2005-04-01,0.005348,0.008279,0.002509,0.001698,-0.013165,0.001045,0.002435,0.000528,0.002787
2005-05-01,0.005417,0.007535,0.003785,0.000773,-0.012991,0.000826,0.001959,0.001469,0.002267
...,...,...,...,...,...,...,...,...,...
2023-08-01,0.008308,0.008803,0.006565,0.008531,0.017963,0.006723,0.005438,0.007914,0.010261
2023-09-01,0.008085,0.009893,0.005953,0.007983,0.016861,0.005830,0.004129,0.007350,0.009086
2023-10-01,0.006929,0.008935,0.005545,0.006251,0.015782,0.004896,0.002798,0.006077,0.008289


In [12]:
# Saving data to xlsx
df_ridge_alpha.to_excel('df_all_alpha_Ridge_Regression.xlsx', index=True)

In [13]:
# Defining alpha model
def Construct_Alpha_Signal_DataFrame(number_of_long_asset,df_alpha):

    df_alpha_signal = pd.DataFrame()
    for i in range(df_alpha.shape[0]):

        signal_date = df_alpha.index[i]    
        date_signal = df_alpha.iloc[i].reset_index(name='alpha').sort_values(by='alpha',ascending=False)
        trade_ticker = pd.DataFrame(date_signal['index'].to_list()[:number_of_long_asset ],index = [ "Top " + str(j+1) + " Alpha" for j in range(number_of_long_asset)], columns=[signal_date]).T
        
        df_alpha_signal  = pd.concat([df_alpha_signal, trade_ticker],axis=0)


        
    df_alpha_signal = df_alpha_signal.shift(1)
    return df_alpha_signal


df_ridge_alpha_signal = Construct_Alpha_Signal_DataFrame(number_of_long_asset=4,df_alpha=df_ridge_alpha)

df_ridge_alpha_signal

,Top 1 Alpha,Top 2 Alpha,Top 3 Alpha,Top 4 Alpha
2005-01-01,None,None,None,None
2005-02-01,XLE,XLF,XLY,XLB
2005-03-01,XLE,XLF,XLB,XLY
2005-04-01,XLE,XLB,XLF,XLY
2005-05-01,XLE,XLB,XLY,XLF
...,...,...,...,...
2023-08-01,XLK,XLY,XLI,XLV
2023-09-01,XLK,XLY,XLE,XLI
2023-10-01,XLK,XLE,XLY,XLB
2023-11-01,XLK,XLE,XLY,XLB


In [14]:
# Saving data to xlsx
df_ridge_alpha_signal.to_excel('df_all_alpha_Ridge_Regression_signal.xlsx', index=True)